In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
import sys
sys.path.append("../../")

In [102]:
import pandas as pd
import geopandas as gpd 
from sklearn.svm import OneClassSVM 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from config.config import BASE_PATH, PATH_TO_PATH_CONFIG_FILE
from src.utils import load_paths_from_yaml, replace_base_path

In [103]:

paths = load_paths_from_yaml(PATH_TO_PATH_CONFIG_FILE)
paths = replace_base_path(paths, BASE_PATH)

In [104]:
def temporal_train_test_split(train_data: pd.DataFrame, date_col: str, train_size: float) -> tuple:
    df = train_data.copy()
    df[date_col] = pd.to_datetime(df.date)
    df = df.sort_values(by=date_col)
    split_index = int(train_size * len(df))
    train_df =  df.iloc[:split_index]
    test_df = df.iloc[split_index:]
    return train_df, test_df

In [124]:
# read in training data
training_data = gpd.read_file(paths["training_data"]["subset"])
training_data.date = pd.to_datetime(training_data.date)

In [128]:
features = ["ffmc", "farmyard_d", "hiking_ds", "forest_ds", "rail_dens", "elevation", "slope", "aspect", "foresttype", "pop_dens"]
training_data = training_data.loc[:, features + ["fire"]]
training_data.dropna(inplace=True)

In [129]:

X = training_data.loc[:, features]
y = training_data.fire

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [130]:
rf = RandomForestClassifier().fit(X_train, y_train)
pred = rf.predict(X_test)

In [133]:
print(accuracy_score(pred, y_test))
print(precision_score(pred, y_test))
print(recall_score(pred, y_test))

0.8571428571428571
0.9459459459459459
0.8898305084745762
